<a href="https://colab.research.google.com/github/DomTHK/DLO_Lab/blob/main/Aufgabe05_MNIST_oldSchool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
device = "cuda" if torch.cuda.is_available() else "cpu"
from torchvision import datasets
from torch.optim import SGD
from timeit import default_timer as timer
#from sklearn.metrics import confusion_matrix
#import seaborn as sn
#import pandas as pd
#from pretty_confusion_matrix import pp_matrix
#from pretty_confusion_matrix import pp_matrix_from_data

Download and load the training data

In [2]:
train_set = datasets.MNIST('data/', download=True, train=True)
train_images = train_set.data #train x
train_targets = train_set.targets #train y

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [3]:
test_set = datasets.MNIST('data/', download=True, train=False)
test_images = test_set.data #test x
test_targets = test_set.targets #test y

In [4]:
??x.view

Object `x.view` not found.


In [5]:
class MNISTDataset(Dataset):
    def __init__(self, x, y):
        x = x.float()/255                 #x normieren
        x = x.view(-1,28*28)              #mache daraus eine liste von vektoren mit der nötigen anzahl an zeilen         
        self.x, self.y = x, y
    def __getitem__(self, ix):
        x, y = self.x[ix], self.y[ix]
        return x.to(device), y.to(device)
    def __len__(self):
        return len(self.x)

In [6]:
def get_data():
    train = MNISTDataset(train_images, train_targets)                     #training data
    train_dl = DataLoader(train, batch_size=42, shuffle=True)             #put in dataloader with batch_size
    test = MNISTDataset(test_images, test_targets)                        #same with testdata
    test_dl = DataLoader(test, batch_size=len(test_images), shuffle=True)
    return train_dl, test_dl #return the dataloaders

In [7]:
def get_model():                                  
    model = nn.Sequential(
        nn.Linear(28 * 28, 90),
        nn.ReLU(),
        nn.Dropout(p=0.12),
        nn.Linear(90, 70),
        nn.ReLU(),
        #nn.Dropout(p=0.2),
        nn.Linear(70, 30),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(30, 10),
    ).to(device)                                   #Schicht mit 3 linearen, 2 sigmoiden (je dazwischen); mit 28^2 -> 40 -> 20 -> 10
    loss_fn = nn.CrossEntropyLoss()                #loss_function: crossentropyloss
    optimizer = SGD(model.parameters(), lr=0.2)   #use SGD with the specified model with lr 0.01
    return model, loss_fn, optimizer

In [8]:
def train_batch(x, y, model, opt, loss_fn):
    model.train()                             #model trainieren
    prediction = model(x)                     #take predictions
    batch_loss = loss_fn(prediction, y)       #given loss-func (here cel) computes loss based on target y and prediction of model
    batch_loss.backward()                     #compute loss backwards
    optimizer.step()                          #make a step
    optimizer.zero_grad()                     #performance boost
    return batch_loss.item()                  #return batch_loss

In [9]:
@torch.no_grad()
def accuracy(x, y, model):                    #
    with torch.no_grad():                     #
        prediction = model(x)                 #get predictions of model
    max_values, argmaxes = prediction.max(-1) #find maximum of predictions
    is_correct = argmaxes == y                #find out if prediction true
    return is_correct.cpu().numpy().tolist()  #return that

In [10]:
@torch.no_grad()
def loss(x, y, model):                       
    prediction = model(x)                    #get predictions of model
    loss = loss_fn(prediction, y)            #calc the loss based on loss function
    return loss.item()                       #return loss

In [11]:
train_dl, test_dl = get_data()
model, loss_fn, optimizer = get_model()

In [12]:
# ----------------------------------------------
# Test >>>
#
#model.load_state_dict(torch.load("nnMnist_40h_20h.ph", map_location='cpu'))

#y_pred = []
#y_true = []

#for data, target in test_dl:
#    data, target = data.to(device), target.to(device)
#    output = model(data)
#    pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
    
#    y_pred.extend(pred.data.cpu().numpy())
#    y_true.extend(target.data.cpu().numpy())

#pp_matrix_from_data(y_true, y_pred, cmap='viridis')

In [ ]:
#----------------------------------------------
# Training >>>
#
print('Starting training...')

time0 = timer()

epochs = 500

arrPlotX = []
train_losses, train_accuracies = [], []
test_losses, test_accuracies = [], []
for epoch in range(epochs):
    train_epoch_losses, train_epoch_accuracies = [], []
    for ix, batch in enumerate(iter(train_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)  
        train_epoch_losses.append(batch_loss)
    train_epoch_loss = np.array(train_epoch_losses).mean()
    for ix, batch in enumerate(iter(train_dl)):
        x, y = batch
        is_correct = accuracy(x, y, model)
        train_epoch_accuracies.extend(is_correct)
    train_epoch_accuracy = np.mean(train_epoch_accuracies)
    for ix, batch in enumerate(iter(test_dl)):
        x, y = batch
        val_is_correct = accuracy(x, y, model)
        validation_loss = loss(x, y, model)
    val_epoch_accuracy = np.mean(val_is_correct)
    arrPlotX.append(epoch)
    train_losses.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    test_losses.append(validation_loss)
    test_accuracies.append(val_epoch_accuracy)
    print( f"epoch: {epoch}  train_acc: {100 * train_epoch_accuracy:.2f}%  test_acc: {100 * val_epoch_accuracy:.2f}%" )
    

Starting training...
epoch: 0  train_acc: 93.83%  test_acc: 93.47%
epoch: 1  train_acc: 95.49%  test_acc: 94.72%
epoch: 2  train_acc: 95.99%  test_acc: 95.66%
epoch: 3  train_acc: 96.89%  test_acc: 96.23%
epoch: 4  train_acc: 96.40%  test_acc: 95.59%
epoch: 5  train_acc: 97.55%  test_acc: 96.48%
epoch: 6  train_acc: 97.74%  test_acc: 96.76%
epoch: 7  train_acc: 97.48%  test_acc: 96.46%
epoch: 8  train_acc: 98.01%  test_acc: 96.79%
epoch: 9  train_acc: 98.12%  test_acc: 96.85%
epoch: 10  train_acc: 98.01%  test_acc: 96.91%
epoch: 11  train_acc: 98.23%  test_acc: 97.07%
epoch: 12  train_acc: 97.92%  test_acc: 96.74%
epoch: 13  train_acc: 97.88%  test_acc: 96.65%
epoch: 14  train_acc: 98.25%  test_acc: 96.89%
epoch: 15  train_acc: 98.61%  test_acc: 96.95%
epoch: 16  train_acc: 98.70%  test_acc: 97.08%
epoch: 17  train_acc: 98.41%  test_acc: 96.89%
epoch: 18  train_acc: 98.73%  test_acc: 97.25%
epoch: 19  train_acc: 98.66%  test_acc: 97.14%
epoch: 20  train_acc: 98.66%  test_acc: 97.19%
ep

In [ ]:
torch.save(model.state_dict(), "nnMnist_40h_20h.ph")

In [ ]:
plt.plot(arrPlotX, train_accuracies)
plt.plot(arrPlotX, test_accuracies)
plt.savefig("accuracies.png")